In [1]:
%load_ext autoreload
%autoreload 2

import pathlib

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.1f' % x)
import scipy.stats
import numpy as np

import plotly.express as px
import plotly.graph_objs as go

from data_handlers.bwm import parse_bwms
# from data_handlers.pidstat import parse_pidstat

In [2]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files/")

In [3]:
name_replacement = {
}

routing_color_map = {
}

In [4]:
def mean_confidence_interval(data, confidence=0.95):
    if isinstance(data.iloc[0], str):
        return data.iloc[0]
    
    a = 1.0 * np.array(data)
    n = len(a)
    h = scipy.stats.sem(a) * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

def error_band_line(error_y_mode=None, **kwargs):
    """Extension of `plotly.express.line` to use error bands."""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.3)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
        # Reorder data as said here: https://stackoverflow.com/a/66854398/8849755
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)
    return fig

In [41]:
network_df = parse_bwms(experiment_path)

Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/359
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/254
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/324
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/353
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/288
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/385
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/266
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/361
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/282
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/316
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/404
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/4/337
Parsing configua

/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/bwm.py:106: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(["Simulation ID", "Software", "CLA", "dt"]).sum().reset_index()


In [191]:
procmem_df = parse_pidstat(experiment_path)

Parsing configuarion 1000, 50, binary_spray in /storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/binary_files/24/ids/1047
Parsing configuarion 1000, 50, cadr_responders in /storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/binary_files/24/ids/983
Parsing configuarion 1000000, 100, prophet in /storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/binary_files/24/ids/1137
Parsing configuarion 1000, 10, cadr_spray in /storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/binary_files/24/ids/910
Parsing configuarion 1000000, 50, cadr_responders in /storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/binary_files/24/ids/967
Parsing configuarion 1000000, 10, binary_spray in /storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/binary_files/24/ids/1075
Parsing configuarion 1000, 50, prophet in /storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/binary_files/24/ids/1196
Parsing configuario

In [42]:
merged = network_df # procmem_df.merge(network_df, left_index=True, right_index=True).reset_index()

In [43]:
network_df

,Software,CLA,Bundles per Second,# Node,# Payloads,Payload Size,dt,bytes_out/s,Mbit/s
0,DTN7Go,mtcp,1,2,1,1000,0.0,0.0,0.0
1,DTN7Go,mtcp,1,2,1,1000,1.0,732.0,0.0
2,DTN7Go,mtcp,1,2,1,1000,2.0,644.0,0.0
3,DTN7Go,mtcp,1,2,1,1000,3.0,720.0,0.0
4,DTN7Go,mtcp,1,2,1,1000,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
307564,Serval,default,100,32,100,1000000,1830.0,168.0,0.0
307565,Serval,default,100,32,100,1000000,1831.0,0.0,0.0
307566,Serval,default,100,32,100,1000000,1832.0,0.0,0.0
307567,Serval,default,100,32,100,1000000,1833.0,0.0,0.0


In [44]:
# cleared = merged[merged["routing"].isin(["epidemic", "cadr_epidemic", "cadr_responders"])]
# cleared.replace(name_replacement, inplace=True)
# cleared['%CPU'] = cleared['%CPU'] / 31

cleared = merged.copy()
cleared["dt"] = pd.to_timedelta(cleared["dt"], unit="S")
cleared.set_index("dt", inplace=True)
cleared_mean = cleared.groupby(["Software", "CLA", "Bundles per Second", "# Node", "# Payloads", "Payload Size"]).resample("1T").mean()
cleared_mean = cleared_mean.drop(["Bundles per Second", "# Node", "# Payloads", "Payload Size"], axis='columns')

cleared_mean = cleared_mean.reset_index()

/tmp/ipykernel_203347/1840908926.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cleared_mean = cleared.groupby(["Software", "CLA", "Bundles per Second", "# Node", "# Payloads", "Payload Size"]).resample("1T").mean()
/tmp/ipykernel_203347/1840908926.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cleared_mean = cleared.groupby(["Software", "CLA", "Bundles per Second", "# Node", "# Payloads", "Payload Size"]).resample("1T").mean()
/tmp/ipykernel_203347/1840908926.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will defau

In [45]:
cleared_mean

,Software,CLA,Bundles per Second,# Node,# Payloads,Payload Size,dt,bytes_out/s,Mbit/s
0,DTN7Go,mtcp,1,2,1,1000,0 days 00:00:00,427.6,0.0
1,DTN7Go,mtcp,1,2,1,100000,0 days 00:00:00,11341.0,0.1
2,DTN7Go,mtcp,1,2,1,1000000,0 days 00:00:00,157762.1,1.2
3,DTN7Go,mtcp,1,2,10,1000,0 days 00:00:00,1556.1,0.0
4,DTN7Go,mtcp,1,2,10,100000,0 days 00:00:00,85368.2,0.7
...,...,...,...,...,...,...,...,...,...
5447,Serval,default,100,32,100,1000000,0 days 00:26:00,59084.1,0.5
5448,Serval,default,100,32,100,1000000,0 days 00:27:00,58801.1,0.4
5449,Serval,default,100,32,100,1000000,0 days 00:28:00,58696.2,0.4
5450,Serval,default,100,32,100,1000000,0 days 00:29:00,59140.6,0.5


In [47]:
fig = error_band_line(
    data_frame=cleared_mean,
    x=cleared_mean["dt"].dt.total_seconds(), 
    y="Mbit/s",
    # error_y=cleared_std["Mbit/s"],
    # error_y_mode="band",
    color="Software",
    line_dash="CLA",
    facet_col="# Payloads",
    facet_row="Payload Size"
)

# fig.layout.margin = go.layout.Margin(l=10, r=10, b=10, t=15)
# fig.update_layout(width=800, height=200, xaxis_range=[0,1])

# fig.write_image("/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/evaluation/paper/figures/cpu.pdf")
fig.show()

In [ ]:
fig = error_band_line(
    data_frame=cleared_mean,
    x=cleared_mean["dt"].dt.total_seconds() / 3600, 
    y="%CPU",
    error_y=cleared_std["%CPU"],
    error_y_mode="band",
    color="routing",
    color_discrete_map=routing_color_map,
    category_orders={
        "routing": ["Epidemic", "ProgDTN Epidemic", "ProgDTN Emergency"],
    },
    labels={
        "x": "Time (h)",
        "routing": "Routing",
        "%CPU": "CPU (%)"
    },
)

fig.layout.margin = go.layout.Margin(l=10, r=10, b=10, t=15)
fig.update_layout(width=800, height=200, xaxis_range=[0,1])

fig.write_image("/storage/research_data/sommer2020cadr/maci-docker-compose/maci_data/evaluation/paper/figures/cpu.pdf")
fig.show()